In [1]:
import pandas as pd
import random
import sqlite3
from sqlite3 import Error

In [2]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(conn)
    except Error as e:
        print(e)
    #finally:
    #    if conn:
    #        conn.close()
    return conn

In [3]:
def run_sql(conn, sql):
    try:
        c = conn.cursor()
        c.execute(sql)
    except Error as e:
        print(e)

In [4]:
conn = create_connection("../data/sqlite.db")

In [5]:
sql_drop_demographics = """
    DROP TABLE IF EXISTS demographics;
"""
run_sql(conn, sql_drop_demographics)

sql_create_demographics = """
    CREATE TABLE IF NOT EXISTS demographics (
        patientuid integer PRIMARY KEY,
        zip_code integer,
        income integer,
        bmi integer
    );
"""
run_sql(conn, sql_create_demographics)

In [6]:
sql_drop_measurements = """
    DROP TABLE IF EXISTS measurements;
"""
run_sql(conn, sql_drop_measurements)

sql_create_measurements = """
    CREATE TABLE IF NOT EXISTS measurements (
        patientuid integer,
        label text,
        value text,
        date text
    );
"""
run_sql(conn, sql_create_measurements)

In [7]:
def create_demographic(conn, params):
    sql = ''' INSERT INTO demographics(patientuid, zip_code, income, bmi) VALUES(?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, params)
    conn.commit()
    return cur.lastrowid

def create_measurement(conn, params):
    sql = ''' INSERT INTO measurements(patientuid, label, value, date) VALUES(?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, params)
    conn.commit()
    return cur.lastrowid

In [8]:
demographics_df = pd.read_csv("../data/demographics.csv")
for i, r in demographics_df.iterrows():
    create_demographic(conn, (int(r['patientuid']), int(r['zip_code']), int(r['income']), int(r['bmi'])))

In [9]:
measurements_df = pd.read_csv("../data/measurements.csv", sep="|")
for i, r in measurements_df.iterrows():
    create_measurement(conn, (int(r['patientuid']), str(r['label']), str(r['value']), str(r['date'])))